In [13]:
import numpy as np

x = np.array([2, 3, 5, 7, 11, 13, 24], dtype=np.uint32)
seq_length = x.shape[0]
largest_element = x.max()
num_total_bits = int(np.ceil(np.log2(largest_element)))
num_lower_bits = int(np.ceil(np.log2(largest_element/seq_length)))

display(num_total_bits, num_lower_bits)


size_lower_bits = 1 + num_lower_bits*seq_length//8
lower_bits = x % (1 << num_lower_bits)

display(lower_bits, size_lower_bits)

5

2

array([2, 3, 1, 3, 3, 1, 0], dtype=uint32)

2

In [19]:
np.binary_repr(np.array(255, dtype=np.uint8) << 1)

'111111110'

In [55]:
import numpy as np
from numpy.typing import NDArray
from typing import Literal

class NDBitstream:
    def __init__(self):
        self.repr_size = 8
        self.num_bits = 0
        self.head_id = 0
        self.bit_arr = np.zeros(10, dtype=np.uint8)

    def __getitem__(self, i: int) -> Literal[0] | Literal[1]:
        return (self.bit_arr[i // 8] >> (7 - (i % 8)) ) % 2

    def append(self, x: np.unsignedinteger) -> None:
        num_bits_to_add = np.ceil(np.log2(x)).astype(np.uint32) + 1
        while num_bits_to_add > 0:
            bits_in_head = self.num_bits % self.repr_size
            bit_space_in_head = self.repr_size - bits_in_head
            lower_bits_of_x_shifted = (x << bits_in_head) & ((1 << self.repr_size) - 1)
            self.bit_arr[self.head_id] += lower_bits_of_x_shifted
            print(bit_space_in_head, num_bits_to_add, self.num_bits, self.head_id, bits_in_head)
            self.num_bits += min(num_bits_to_add, self.repr_size)
            num_bits_to_add -= bit_space_in_head 
            self.head_id = self.num_bits // 8


arr = np.array([2, 3, 5, 7, 11, 13, 24], dtype=np.uint8)
bitstream = NDBitstream()
bitstream.append(12)
bitstream.bit_arr

IndentationError: expected an indented block after 'for' statement on line 16 (1830643465.py, line 18)

In [122]:
import numpy as np
from numpy.typing import NDArray
from typing import Literal

def elias_fano_encode(arr: NDArray[np.uint32]) -> tuple[NDArray[np.uint8], NDArray[np.uint8]]:
    r = repr_size = 8
    assert arr.ndim == 1
    n = len(arr)
    m = arr.max()
    b = int(np.ceil(np.log2(m)))
    l = int(np.ceil(np.log2(m/n)))

    # compute lower bits
    mask_lower_bits = ((1 << l) - 1)
    lower_bits = arr & mask_lower_bits 
    total_low_bits = l*n
    L = np.zeros(1 +  total_low_bits // r, dtype=np.uint8)
    bits_in_head = l*n % r

    for i in range(n):
        # print(i, l*i // r, l*i % r, lower_bits[i])
        L[l*i // r] += lower_bits[i] << r - l - (l*i % r)

    # compute upper bits
    num_buckets = 1 << (b - l)
    upper_bits = arr >> l
    bucket_counts = np.zeros(num_buckets)
    buckets, _bucket_counts = np.unique(upper_bits, return_counts=True)
    bucket_counts[buckets] = _bucket_counts
    total_high_bits = num_buckets + n
    H = np.zeros(1 + total_high_bits // r)

    for i in range(num_buckets):
        ones_to_add = bucket_counts[i]
    return H, L, bits_in_head

def elias_fano_decode(H: NDArray[np.uint8], L: NDArray[np.uint8], n: int, bits_in_head: int) -> NDArray[np.uint32]:
    r = repr_size = 8
    # assert H.ndim == 1
    assert L.ndim == 1

    l = (r*(L.shape[0] - 1) + bits_in_head) // n
    lower_bits = np.zeros(n, dtype=np.uint32)
    for i in range(n):
        # print(i, l*i // r, l*i % r, lower_bits[i])
        # print(L[l*i // r], r - l - (l*i % r), L[l*i // r] >> r - l - (l*i % r))
        lower_bits[i] = (L[l*i // r] >> r - l - (l*i % r)) & ((1 << l) - 1)
    return lower_bits



arr = np.array([2, 3, 5, 7, 11, 13, 24], dtype=np.uint8)
H, L, bits_in_head = elias_fano_encode(arr)
display(H, L, bits_in_head)
# display(L)
# display(np.binary_repr(L[0], width=8))
arr_decoded = elias_fano_decode(L=L, H=None, bits_in_head=bits_in_head, n=len(arr))
# display(arr_decoded)
# display(efr)
# for x in efr:
#     print(np.binary_repr(x, width=8))

(array([0, 1, 2, 3, 6], dtype=uint8), array([2, 2, 1, 1, 1]))

array([183, 208], dtype=uint8)

6

In [18]:
import numpy as np
from numpy.typing import NDArray

def elias_fano_encode(arr: NDArray[np.uint32]) -> tuple[NDArray[np.uint8], NDArray[np.uint8]]:
    assert arr.ndim == 1
    n = len(arr)
    m = arr.max()
    b = int(np.ceil(np.log2(m)))
    l = int(np.ceil(np.log2(m/n)))

    print('Upper bound to bytes-per-id:', (2*n + n*l)/8/n)

    arr_unpacked = np.unpackbits(arr[None].T.view(np.uint8), axis=1, bitorder='little')
    lower_bits = np.packbits(arr_unpacked[:, :l], bitorder='little')

    # compute upper bits
    num_buckets = 1 << (b - l)
    upper_bits = arr >> l
    bucket_counts = np.zeros(num_buckets)
    buckets, _bucket_counts = np.unique(upper_bits, return_counts=True)
    bucket_counts[buckets] = _bucket_counts

    upper_bits = list()
    for c in bucket_counts:
        upper_bits += int(c)*[1] + [0]
    upper_bits = np.packbits(np.array(upper_bits), bitorder='little')
    return upper_bits, lower_bits

    # compute lower bits
    mask_lower_bits = ((1 << l) - 1)
    lower_bits = arr & mask_lower_bits 
    total_low_bits = l*n
    L = np.zeros(1 +  total_low_bits // r, dtype=np.uint8)
    bits_in_head = l*n % r

    for i in range(n):
        # print(i, l*i // r, l*i % r, lower_bits[i])
        L[l*i // r] += lower_bits[i] << r - l - (l*i % r)

    # compute upper bits
    num_buckets = 1 << (b - l)
    upper_bits = arr >> l
    bucket_counts = np.zeros(num_buckets)
    buckets, _bucket_counts = np.unique(upper_bits, return_counts=True)
    bucket_counts[buckets] = _bucket_counts
    total_high_bits = num_buckets + n
    H = np.zeros(1 + total_high_bits // r)

    for i in range(num_buckets):
        ones_to_add = bucket_counts[i]
    return H, L, bits_in_head

# arr = np.array([2, 3, 5, 7, 11, 13, 24], dtype=np.uint32)
arr = np.sort(np.random.randint(0, 2**32 - 1, size=2_000))
H, L = elias_fano_encode(arr)
(H.nbytes + L.nbytes)/len(arr)

Upper bound to bytes-per-id: 3.0


2.939

In [34]:
import numpy as np

np.unpackbits(np.array([256 + 255 - 128], dtype=np.uint16).view(np.uint8), bitorder='little')

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)